In [8]:
# 加油！！！
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

x, y = datasets.make_moons(n_samples=50000, noise=0.3, random_state=42)
print(x.shape)
print(y)

# plt.scatter(x[y == 0, 0], x[y == 0, 1])  # 标签为0的样本中第一列特征数据作为横坐标第二列特征数据作为纵坐标
# plt.scatter(x[y == 1, 0], x[y == 1, 1])  # 标签为1的样本中第一列特征数据作为横坐标第二列特征数据作为纵坐标
# plt.show()
# 默认分割比例是75%和25% test_size=0.25
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)
log_clf = LogisticRegression()
log_clf.fit(x_train, y_train)
print(log_clf.score(x_test, y_test))

svm_clf = SVC()
svm_clf.fit(x_train, y_train)
print(svm_clf.score(x_test, y_test))

df_clf = DecisionTreeClassifier()
df_clf.fit(x_train, y_train)
print(df_clf.score(x_test, y_test))

y_predict1 = log_clf.predict(x_test)
y_predict2 = svm_clf.predict(x_test)
y_predict3 = df_clf.predict(x_test)

(50000, 2)
[1 0 0 ... 1 0 0]
0.85336
0.91216
0.87544


In [9]:
# 手动实现bagging原理
y_predict = np.array((y_predict1 + y_predict2 + y_predict3) >= 2, dtype='int')
print(y_predict[:10])
print(y_test[:10])


[1 1 1 0 0 0 0 0 0 1]


array([1, 1, 1, 0, 0, 0, 0, 1, 0, 1], dtype=int64)

In [10]:
from sklearn.ensemble import VotingClassifier

#硬投票无权值
voting_clf = VotingClassifier(
    estimators=[('log_clf', LogisticRegression()), ('svm_clf', SVC()), ('dt_clf', DecisionTreeClassifier())],
    voting='hard')
voting_clf.fit(x_train, y_train)
print(voting_clf.score(x_test, y_test))

0.9044

In [12]:
#硬投票无权值
voting_clf = VotingClassifier(
    estimators=[('log_clf', LogisticRegression()), ('svm_clf', SVC(probability=True)),  #SVC默认不输出概率值
                ('dt_clf', DecisionTreeClassifier())],
    voting='soft')
voting_clf.fit(x_train, y_train)
print(voting_clf.score(x_test, y_test))

0.90376


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [18]:
% % time
#cpu time是实际cpu运行算法时间，wall time是墙上真实时间，就是real time（包含了进程启动时间，I/O等待时间等）
#有放回抽样bootstrap
bagging_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500, max_samples=2000, bootstrap=True)
bagging_clf.fit(x_train, y_train)
print(bagging_clf.score(x_test, y_test))

0.91224
CPU times: total: 3.22 s
Wall time: 3.26 s


In [19]:
% % time
#n_jobs使用多核的耗时更少
bagging_clf = BaggingClassifier(DecisionTreeClassifier(),
                                n_estimators=500, max_samples=2000,
                                bootstrap=True, n_jobs=-1)
bagging_clf.fit(x_train, y_train)
bagging_clf.score(x_test, y_test)

CPU times: total: 391 ms
Wall time: 2.95 s


0.91336

In [20]:
#bootstrap_features  对特征进行采样
#因只有两个特征 较少 max_features设置为1
#oob_score_ out of bag 不使用测试数据集，而是使用没有取到的样本作为测试/验证
random_subspaces_clf = BaggingClassifier(DecisionTreeClassifier(),
                                         n_estimators=500, max_samples=500, oob_score=True,
                                         bootstrap=True, n_jobs=-1,
                                         max_features=1, bootstrap_features=True)
random_subspaces_clf.fit(x, y)
print(random_subspaces_clf.oob_score_)

0.88824


In [21]:
# 总的抽样的样本数需小于训练集的数目
# 也可叫随机森林
random_patches_clf = BaggingClassifier(DecisionTreeClassifier(),
                                       n_estimators=500, max_samples=40000, oob_score=True,
                                       bootstrap=True, n_jobs=-1,
                                       max_features=1, bootstrap_features=True)
random_patches_clf.fit(x, y)  #整个样本作为训练集
print(random_patches_clf.oob_score_)

0.81756


In [22]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
% % time
rf_clf = RandomForestClassifier(n_estimators=500, random_state=666, oob_score=True, n_jobs=-1)
rf_clf.fit(x, y)
print(rf_clf.oob_score_)


0.90322
CPU times: total: 41.6 s
Wall time: 8.05 s


In [ ]:
% % time
rf_clf = RandomForestClassifier(max_samples=37500, n_jobs=-1)
rf_clf.fit(x_train, y_train)
print(rf_clf.score(x_test, y_test))

In [27]:
% % time
rf_clf2 = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=666, oob_score=True, n_jobs=-1)
rf_clf2.fit(x, y)
print(rf_clf2.oob_score_)

0.90962
CPU times: total: 22.5 s
Wall time: 4.85 s


In [28]:
% % time
#Extra-Trees
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(n_estimators=500, oob_score=True, bootstrap=True, n_jobs=-1)
et_clf.fit(x, y)
print(et_clf.oob_score_)

0.90656
CPU times: total: 15.8 s
Wall time: 5.79 s


In [ ]:
#Boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
% % time
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=500)
ada_clf.fit(x_train, y_train)
print(ada_clf.score(x_test, y_test))

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(max_depth=2, n_estimators=500)
gb_clf.fit(x_train, y_train)
print(gb_clf.score(x_test, y_test))

0.91144
